In [3]:
import pandas as pd
import os
import numpy as np


In [5]:
file_path = r"F:\嘉兴移动\政企\物联网\材料\2025\经分会\8月\\"

try:
    # 读取Excel文件
    july_data = pd.read_excel(os.path.join(file_path, "2025年政企5G物联网转型产品等效发展7月.xlsx"))
    august_data = pd.read_excel(os.path.join(file_path, "2025年政企5G物联网转型产品等效发展8月.xlsx"))

    # 打印列名以便确认
    print("7月数据的列名:", july_data.columns.tolist())
    print("8月数据的列名:", august_data.columns.tolist())

    # 确保网格列是字符串类型
    july_data['网格'] = july_data['网格'].astype(str).str.strip()
    august_data['网格'] = august_data['网格'].astype(str).str.strip()

    # 处理7月数据
    july_summary = july_data.groupby('网格')['折算后条数'].sum().reset_index()
    july_summary.rename(columns={'折算后条数': '7月'}, inplace=True)

    # 处理8月数据
    august_summary = august_data.groupby('网格')['折算后条数'].sum().reset_index()
    august_summary.rename(columns={'折算后条数': '8月'}, inplace=True)

    # 合并数据
    merged_data = pd.merge(july_summary, august_summary, on='网格', how='outer')

    # 添加网格一致性检查列
    merged_data['两个月网格不一致'] = '一致'

    # 检查哪些网格只在一个月中存在
    july_grids = set(july_summary['网格'])
    august_grids = set(august_summary['网格'])

    if july_grids != august_grids:
        only_in_july = july_grids - august_grids
        only_in_august = august_grids - july_grids

        # 更新不一致的标记
        merged_data.loc[merged_data['网格'].isin(only_in_july), '两个月网格不一致'] = '仅7月存在'
        merged_data.loc[merged_data['网格'].isin(only_in_august), '两个月网格不一致'] = '仅8月存在'

        print("警告：7月和8月的网格不一致！")
        print(f"仅在7月中存在的网格: {only_in_july}")
        print(f"仅在8月中存在的网格: {only_in_august}")

    # 填充NaN值为0
    merged_data['7月'] = merged_data['7月'].fillna(0)
    merged_data['8月'] = merged_data['8月'].fillna(0)

    # 定义网格类别
    class1_grids = [
        "嘉禾经开", "桐乡梧桐", "嘉善罗星", "桐乡濮院", "嘉禾秀洲",
        "海宁硖石", "平湖当湖", "海盐武原", "嘉禾南湖", "海宁许村"
    ]

    class2_grids = [
        "海宁盐官", "嘉禾洪合", "平湖经开", "桐乡洲泉", "嘉禾王江泾",
        "嘉善魏塘", "嘉善经开", "桐乡崇福", "平湖乍浦", "海宁长安",
        "桐乡乌镇", "嘉禾王店", "嘉禾城东", "桐乡开发区", "嘉禾城北",
        "海宁海昌", "嘉禾大桥", "嘉禾余新"
    ]

    class3_grids = [
        "平湖新埭", "嘉善姚庄", "嘉善西塘", "桐乡大麻", "嘉善干窑",
        "海盐百步", "海宁斜桥", "桐乡屠甸", "海宁马桥", "海盐秦山",
        "平湖新仓", "平湖独山港", "海宁袁花", "海盐通元", "桐乡石门",
        "嘉禾油车港", "海盐经开", "嘉禾新丰"
    ]

    # 添加网格类别列
    def classify_grid(grid_name):
        # 确保比较时去掉"网格"二字（如果存在）
        clean_name = grid_name.replace('网格', '').strip()

        if clean_name in class1_grids:
            return '一类网格'
        elif clean_name in class2_grids:
            return '二类网格'
        elif clean_name in class3_grids:
            return '三类网格'
        else:
            return '其他网格'

    merged_data['网格类别'] = merged_data['网格'].apply(classify_grid)

    # 按网格类别和网格名称排序
    merged_data = merged_data.sort_values(['网格类别', '网格'])

    # 保存结果
    output_file = os.path.join(file_path, "数据汇总.xlsx")
    merged_data.to_excel(output_file, index=False)

    print(f"数据汇总完成！结果已保存到: {output_file}")
    print(f"总共处理了 {len(merged_data)} 个网格的数据")

    # 统计各类别网格数量
    print("\n网格类别统计:")
    category_counts = merged_data['网格类别'].value_counts()
    for category, count in category_counts.items():
        print(f"{category}: {count}个网格")

    print("\n数据预览:")
    print(merged_data.head(10))

except Exception as e:
    print(f"发生错误: {e}")
    print("请检查文件路径和列名是否正确")

7月数据的列名: ['数据截至日期', '月份', '县市', '网格', '集团编号', '集团名称', '集团级别', '集团状态', '客户经理ID', '客户经理', '是否重客', '业务大类', '业务', '数量', '系数', '折算后条数']
8月数据的列名: ['数据截至日期', '月份', '县市', '网格', '集团编号', '集团名称', '集团级别', '集团状态', '客户经理ID', '客户经理', '是否重客', '业务大类', '业务', '数量', '系数', '折算后条数']
警告：7月和8月的网格不一致！
仅在7月中存在的网格: set()
仅在8月中存在的网格: {'平湖新埭网格'}
数据汇总完成！结果已保存到: F:\嘉兴移动\政企\物联网\材料\2025\经分会\8月\\数据汇总.xlsx
总共处理了 47 个网格的数据

网格类别统计:
三类网格: 18个网格
二类网格: 18个网格
一类网格: 10个网格
其他网格: 1个网格

数据预览:
        网格      7月      8月 两个月网格不一致  网格类别
3   嘉善罗星网格   73.54  228.25       一致  一类网格
7   嘉禾南湖网格   49.93  142.51       一致  一类网格
16  嘉禾秀洲网格   60.24  190.48       一致  一类网格
17  嘉禾经开网格   55.81  142.15       一致  一类网格
19  平湖当湖网格   31.00   67.00       一致  一类网格
29  桐乡梧桐网格  122.80  229.10       一致  一类网格
31  桐乡濮院网格   54.73   91.69       一致  一类网格
36  海宁硖石网格   37.37   84.70       一致  一类网格
38  海宁许村网格   51.43  139.52       一致  一类网格
41  海盐武原网格   10.43   14.69       一致  一类网格
